In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline
pd.set_option('display.max_columns', 100)

In [2]:
home30day = pd.read_csv('../data/home30day.csv',low_memory=False)
university = pd.read_csv('../data/university.csv',low_memory=False)

In [3]:
home30day_5 = home30day.head(5)
home30day_5_grouped = home30day_5.groupby('state')
university_grouped = university.groupby('state')
dfMin = pd.DataFrame()

In [4]:
saleprice = home30day['saleprice'].unique()
print len(saleprice)
print time.time()

10312
1477871993.71


In [5]:
home30day.describe()

,latitude,longitude,numbed,num_bath_full,num_bath_part,rentalprice_min,rentalprice_max,saleprice,floor_plan,size,pool,gatedCommunity,stainlessAppliances,fireplace,renovation
count,183460.000000,183460.000000,183460.000000,183460.000000,183460.000000,183460.000000,183460.0,1.834600e+05,0.0,183460.000000,183460.000000,183460.000000,183460.000000,183460.000000,183460.000000
mean,33.545487,-93.689711,1.966156,1.393595,-0.697100,984.602317,-1.0,1.338273e+05,NaN,1222.413938,0.221765,0.068609,0.099280,0.242985,0.089807
std,5.212622,14.886998,1.897425,1.669656,0.716976,3306.473516,0.0,6.412166e+05,NaN,1223.742908,0.415435,0.252789,0.299039,0.428887,0.285906
min,19.090885,-161.786590,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000e+00,NaN,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.361464,-94.455809,1.000000,1.000000,-1.000000,-1.000000,-1.0,-1.000000e+00,NaN,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.323529,-87.718835,3.000000,2.000000,-1.000000,-1.000000,-1.0,0.000000e+00,NaN,1157.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,36.088063,-85.468457,3.000000,2.000000,-1.000000,1395.000000,-1.0,1.350000e+05,NaN,1812.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,64.978010,-70.718870,12.000000,47.000000,1.000000,200000.000000,-1.0,1.500000e+08,NaN,30000.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
def distance(x1,x2,y1,y2):
    return np.linalg.norm(np.array([x1,y1]) - np.array([x2,y2]))

def nearestPair(homes,universities):
    minDistPair = pd.DataFrame()
    states = home['state'].unique()
#     states = ['CA','AL','LA']
    homes_grouped = homes.groupby('state')
    university_grouped = universities.groupby('state')
    for state in states:
        print state
        home_state = homes_grouped.get_group(state)
        university_state = university_grouped.get_group(state)
        """
        home_city = home_state[home_state['city'].isin(university_state['city'])]
        home_nocity = home_state[~home_state['city'].isin(university_state['city'])]
        home_nocity_zip = home_nocity[home_nocity['zipcode'].isin(university_state['postcode'])]
        home_nocity_nozip = home_nocity[~home_nocity['zipcode'].isin(university_state['postcode'])]
        
        pair_city = nearest(home_city, university_state, 'city', 'city')
        #pair_zip = nearest(home_nocity_zip, university_state, 'zipcode', 'postcode')
        pair_general = nearest(home_nocity_nozip, university_state, 'state', 'state')
        """
        minDistPair_state = nearestInState(home_state, university_state)
        minDistPair = pd.concat([minDistPair, minDistPair_state])
        #print state
        #print home_state.shape
        #print university_state.shape
        #minDistPair_state = nearestInState(home_state, university_state)
        #minDistPair = pd.concat([minDistPair, minDistPair_state])
        #print state, minDistPair_state.shape, home_state.shape
        
    return minDistPair

def nearestInState(h, u):
    # pair a house to every university in the same state
    mergeDf = h.merge(u, left_on='state', right_on='state', how='inner')
    # find distance between a house and an university
    mergeDf['distance'] = mergeDf.apply(lambda row: distance(row['longitude_x'],row['longitude_y'],row['latitude_x'],row['latitude_y']), axis=1)
    # find the distance from a house to the nearest university
    mergeDfGrouped = mergeDf.groupby(['latitude_x','longitude_x'],as_index=False)
    #print len(mergeDfGrouped)
    #print mergeDf.shape
    #print list(mergeDf.columns.values)
    # filter the house-university pairs by the smallest distance found above
    minDistPair = pd.DataFrame()
    for name, group in mergeDfGrouped:
        pair = mergeDf[mergeDf['distance'] == mergeDfGrouped.get_group(name).min()['distance']].head(1)
        minDistPair = pd.concat([minDistPair, pair])
    #mergeDfFinal = mergeDf[mergeDf['latitude_x'].isin(mergeDfMin['latitude_x']) & mergeDf['longitude_x'].isin(mergeDfMin['longitude_x'])]
    #print minDistPair.shape
    return minDistPair
    
    #print mergeDf['distance'].unique()
    
home = home30day

allPair = nearestPair(home,university).reset_index(drop=True)
allPair.shape

CA
AL
TN
LA
DE
DC
CO
AR
FL
MN
MO
AZ
OR
IN
UT
MI
RI
NV
NH
AK
WY
HI
VT


(149348, 56)

In [ ]:
len(home['state'].unique())

In [ ]:
allPair.head(2)

In [7]:
states = home30day['state'].unique()
homes_grouped = home30day.groupby('state')
university_grouped = university.groupby('state')
vthome = homes_grouped.get_group('VT')
univt = university_grouped.get_group('VT')
vt_zip = vthome[vthome['zipcode'].isin(univt['postcode'])]
vt_nozip = vthome[~vthome['zipcode'].isin(univt['postcode'])]
print vthome.shape, vt_zip.shape, vt_nozip.shape

(256, 39) (58, 39) (198, 39)


In [8]:
vt_city = vthome[vthome['city'].isin(univt['city'])]
vt_nocity = vthome[~vthome['city'].isin(univt['city'])]
print vthome.shape, vt_city.shape, vt_nocity.shape

(256, 39) (93, 39) (163, 39)


In [15]:
print list(allPair.columns.values)
print allPair.shape
print allPair.info()

['website_x', 'home_url', 'property_type', 'record_type', 'parser_type', 'latitude_x', 'longitude_x', 'streetaddr_x', 'city_x', 'state', 'zipcode', 'country', 'numbed', 'num_bath_full', 'num_bath_part', 'rentalprice_min', 'rentalprice_max', 'saleprice', 'yearbuilt', 'floor_plan', 'garage', 'lotsize', 'stories', 'size', 'pool', 'style', 'numofparking', 'gatedCommunity', 'stainlessAppliances', 'fireplace', 'floorType', 'kitchenCountertop', 'renovation', 'elevation', 'elevationType', 'description', 'crawl_time', 'datalisted', 'inactive', 'UniversityName', 'streetaddr_y', 'city_y', 'postcode', 'acceptanceRate', 'ranking', 'enroll', 'latitude_y', 'longitude_y', 'type', 'endowment', 'acadStaff', 'students', 'underGrad', 'postGrad', 'website_y', 'distance']
(149348, 56)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149348 entries, 0 to 149347
Data columns (total 56 columns):
website_x              149348 non-null object
home_url               149343 non-null object
property_type          

In [11]:
corr = allPair.corr()
price_corr = corr["saleprice"]
price_corr = price_corr.to_dict()
# print price_corr
for ele in sorted(price_corr.items(), key = lambda x: -abs(x[1])):
    print("{0}: \t{1}".format(*ele))

size: 	0.212434739444
num_bath_part: 	0.0259196241384
rentalprice_max: 	nan
floor_plan: 	nan
saleprice: 	1.0
fireplace: 	0.114797524331
longitude_y: 	-0.101855836736
num_bath_full: 	0.101698415286
longitude_x: 	-0.101392110915
numbed: 	0.0905193630614
pool: 	0.058742727049
rentalprice_min: 	-0.0545353930517
gatedCommunity: 	0.0460546172569
latitude_y: 	-0.0272588998541
latitude_x: 	-0.0272049802005
stainlessAppliances: 	0.0259795581264
renovation: 	0.0128040081913
distance: 	0.00965132092317


In [16]:
print corr.shape

(18, 18)


In [17]:
import seaborn as sns
sns.heatmap(corr, vmax=1, square=True)

ImportError: No module named seaborn